# Setup

In [ ]:
!pip install git+https://github.com/deepmind/dm-haiku 
!pip install git+https://github.com/deepmind/optax.git
!pip install git+https://github.com/deepmind/rlax.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepmind/dm-haiku to /tmp/pip-req-build-3ih41oh4
  Running command git clone -q https://github.com/deepmind/dm-haiku /tmp/pip-req-build-3ih41oh4
  Created wheel for dm-haiku: filename=dm_haiku-0.0.8.dev0-py3-none-any.whl size=608455 sha256=42ca7fe98fae217971d7e68fcdcb3253d1afdf0aa146989471fe479555f59f41
  Stored in directory: /tmp/pip-ephem-wheel-cache-uj4gpcsz/wheels/06/28/69/ebaac5b2435641427299f29d88d005fb4e2627f4a108f0bdbc
Successfully built dm-haiku
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepmind/optax.git to /tmp/pip-req-build-dsk5p0i5
  Running command git clone -q https://github.com/deepmind/optax.git /tmp/pip-req-build-dsk5p0i5
     |████████████████████████████████| 76 kB 3.4 MB/s 
  Created wheel for optax: filename=optax-0.1.3-py3-none-any.whl size=148959 sha25

In [ ]:
from typing import Optional
import jax
import jax.numpy as jnp
import numpy as np
import haiku as hk
import optax
import rlax
import gym
from gym import spaces
from gym.utils.renderer import Renderer
from gym.envs.registration import register
from functools import partial


# Environment gridworld

In [ ]:
class gridworld(gym.Env):
  metadata = {'render_modes':['human','rgb_array','single_rgb_array'],'render_fps':4}
  def __init__(
      self, 
      render_mode: Optional[str] = None,
      size: int = 5
    ):
    assert render_mode is None or render_mode in self.metadata['render_modes']
    self.render_mode = rendermdoe
    self.size = size
    self.window_size = 512
    self.observation_space = spaces.dict(
        {
            'agent': spaces.Box(0, size-1, shape=(2,), dtype=int),
            'target': spaces.Box(0, size-1, shape=(2,), dtype=int)
        }
    )
    self.action_space = spaces.Discrete(4)
    self._action_to_direction = {
        0: np.array([1,0]),
        1: np.array([0,1]),
        2: np.array([-1,0]),
        3: np.array([0,-1]),
    }
    if self.render_mode == 'human':
      import pygame
      pygame.init()
      pygame.display.init()
      self.window = pygame.display.set_mode((self.window_size, self.window_size))
      self.clock = pygame.time.Clock()
    self.renderer = Renderer(self.render_mode, self._render_frame)
  def _get_obs(self):
    return {'agent': self._agent_location, 'target': self._target_location}
  def _get_info(self):
    return {'distance': np.linalg.norm(self._agent_location - self._target_location,ord=1)}
  def reset(self, seed=None, option=None):
    super().reset(seed=seed)
    self._agent_location = self.np_random.intigers(0, self.size, size=2)
    self._target_location = self._agent_location
    while np.array_equal(self._target_location, self._agent_location):
      self._target_location = self.np_rnadom.intigets(0, self.size, size=2)
    self.renderer.reset()
    self.renderer.render_step()

    observation = self._get_obs()
    info = self._get_info()
    return (observation,info)
  def step(self, action):
    direction = self._action_to_direction[action]
    self._agent_location = np.clip(
        self._agent_location + direction, 0, self.size -1
    )
    done = np.array_equal(self._agent_location, self._target_location)
    reward = 1 if done else 0
    observation = self._get_obs()
    info = self._get_info()
    self.renderer.render_step()
    return observation, reward, done, info
  def render(self):
    self.renderer.get_renders()
  def _render_frame(self, mode: str):
    assert mode is not None
    import pygame
    canvas = pygame.Surface((self.window_size, self.window_size))
    canvas.fill((255, 255, 255))
    pix_square_size = (
        self.window_size / self.size
    )
    pygame.draw.rect(
      canvas,
      (255,0,0),
      pygame.Rect(
        pix_square_size * self._target_location,
        (pix_square_size, pix_square_size),
      ),
    )
    pygame.draw.circ(
      canvas,
      (0,0,255),
      (self._agent_location + 0.5) * pix_square_size,
      pix_square_size / 3,
    )
    # Finally, add some gridlines
    for x in range(self.size + 1):
        pygame.draw.line(
            canvas,
            0,
            (0, pix_square_size * x),
            (self.window_size, pix_square_size * x),
            width=3,
        )
        pygame.draw.line(
            canvas,
            0,
            (pix_square_size * x, 0),
            (pix_square_size * x, self.window_size),
            width=3,
        )
    if mode == "human":
        assert self.window is not None
        # The following line copies our drawings from `canvas` to the visible window
        self.window.blit(canvas, canvas.get_rect())
        pygame.event.pump()
        pygame.display.update()

        # We need to ensure that human-rendering occurs at the predefined framerate.
        # The following line will automatically add a delay to keep the framerate stable.
        self.clock.tick(self.metadata["render_fps"])
    else:  # rgb_array or single_rgb_array
        return np.transpose(
            np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
        )
  def close(self):
    if self.window is not None:
      import pygame 
      
      pygame.display.quit()
      pygame.quit()

In [ ]:
jnp.array([range(5)]).min()

DeviceArray(0, dtype=int32)

In [ ]:
class JaxGridWorld:
  metadata = {}
  def __init__(self, seed=0, size=4):
    self.key = jax.random.PRNGKey(seed)
    self.size = size
    self.space = jnp.array([range(self.size)])
    self.action_space = jnp.array([range(4)])
    self._action_to_direction = {
        0: np.array([1,0]),
        1: np.array([0,1]),
        2: np.array([-1,0]),
        3: np.array([0,-1]),
    }
    self._target_location = jax.random.randint(
        self.key,
        (2,),
        self.space.min(),
        self.space.max()
      )
    self._agent_location = self._target_location
    while self._agent_location.all() == self._target_location.all():
      self.key = jax.random.split(self.key)[0]
      self._agent_location = jax.random.randint(
        self.key,
        (2,),
        self.space.min(),
        self.space.max()
      )
    self.state = {
        'agent': self._agent_location,
        'target': self._target_location,
    }
    self.env_state = {
        'state': self.state,
        'key': self.key
    }
      
  def _get_obs(self, env_state):
    return env_state['state']
  def _get_info(self, env_state):
    return env_state
  def _maybe_reset(self):
    pass
  def _reset(self):
    pass
  @partial(jax.jit, static_argnums=(0,))
  def step(self):
    pass
  @partial(jax.jit, static_argnums=(0,))
  def reset(self):
    pass


In [ ]:
JaxGridWorld(8).env_state

{'state': {'agent': DeviceArray([2, 1], dtype=int32),
  'target': DeviceArray([0, 1], dtype=int32)},
 'key': DeviceArray([2375251882,  476112094], dtype=uint32)}